In [354]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd

In [355]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/Bankloan.csv', 
                   encoding = 'cp1251', 
                   sep = ';', decimal = ',')

In [356]:
# выводим первые 5 наблюдений датафрейма
data.head()

,age,job,employ,address,income,debtinc,creddebt,othdebt,default
0,28,working - other,7,2,44,17.7,2.990592,4.797408,0
1,64,working - production,34,17,116,14.7,5.047392,12.004608,0
2,40,working - IT,20,12,61,4.8,1.042368,1.885632,0
3,30,working - IT,11,3,27,34.5,1.751220,7.563780,0
4,25,working - IT,2,2,30,22.4,0.759360,5.960640,1


In [357]:
# смотрим типы переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 9 columns):
age         1500 non-null int64
job         1500 non-null object
employ      1500 non-null int64
address     1500 non-null int64
income      1500 non-null int64
debtinc     1500 non-null float64
creddebt    1500 non-null float64
othdebt     1500 non-null float64
default     1500 non-null int64
dtypes: float64(3), int64(5), object(1)
memory usage: 105.5+ KB


In [358]:
# создаем массив признаков и массив меток
y = data.pop('default')
X = pd.get_dummies(data)

## Случайное разбиение на обучающую и тестовую выборки

In [359]:
# импортируем из модуля sklearn.model_selection 
# функцию train_test_split()
from sklearn.model_selection import train_test_split
# выполняем разбиение на обучающую и тестовую выборки
X_tr, X_tst, y_tr, y_tst = train_test_split(X, y, 
                                            test_size=0.3, 
                                            random_state=42)

In [360]:
# импортируем класс DecisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier
# создаем экземпляр класса DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=152)
# обучаем модель
tree.fit(X_tr, y_tr)
# оцениваем качество модели на тестовых данных
print('Правильность на тестовой выборке: {:.3f}'.format(
    tree.score(X_tst, y_tst)))

Правильность на тестовой выборке: 0.647


## Обычная k-блочная перекрестная проверка

In [361]:
# импортируем функцию cross_val_score() и класс StratifiedKFold
from sklearn.model_selection import cross_val_score, StratifiedKFold
# создаем экземпляр класса StratifiedKFold
strat = StratifiedKFold(n_splits=10, shuffle=True, 
                        random_state=42)
# создаем экземляр класса DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=152)
# вычисляем значение правильности, усредненное по проверочным
# блокам обычной перекрестной проверки
scores_acc_tr = cross_val_score(tree, 
                                X, 
                                y, 
                                cv=strat)
print('Среднее значение правильности: {:.2f}'.format(
    scores_acc_tr.mean()))

Среднее значение правильности: 0.68


In [362]:
# вычисляем значение правильности, усредненное по
# проверочным блокам повторной 
# перекрестной проверки
scores_auc_tr = cross_val_score(tree, 
                                X, y, 
                                scoring='roc_auc', 
                                cv=strat)
print('Среднее значение AUC: {:.2f}'.format(
    scores_auc_tr.mean()))

Среднее значение AUC: 0.65


## Повторная k-блочная перекрестная проверка

In [363]:
# импортируем класс RepeatedKFold
from sklearn.model_selection import RepeatedKFold
# создаем экземпляр класса RepeatedKFold
rkf = RepeatedKFold(n_splits=10, n_repeats=10, 
                    random_state=42)
# вычисляем значение правильности, усредненное по проверочным
# блокам повторной перекрестной проверки
scores_acc_tr = cross_val_score(tree, 
                                X, 
                                y, 
                                cv=rkf)
print('Среднее значение правильности: {:.2f}'.format(
    scores_acc_tr.mean()))

Среднее значение правильности: 0.68


## Перекрестная проверка с исключением по одному

In [364]:
# импортируем класс LeaveOneOut
from sklearn.model_selection import LeaveOneOut
# создаем экземпляр класса LeaveOneOut 
loo = LeaveOneOut()
# вычисляем значение правильности, усредненное по проверочным
# блокам перекрестной проверки c исключением по одному
scores_acc_tr = cross_val_score(tree, 
                                X, 
                                y, 
                                cv=loo)
print('Количество итераций cv: ', len(scores_acc_tr))
print('Среднее значение правильности: {:.2f}'.format(
    scores_acc_tr.mean()))

Количество итераций cv:  1500
Среднее значение правильности: 0.67


## Комбинированная проверка в ручном режиме

In [365]:
# импортируем функцию train_test_split()
from sklearn.model_selection import train_test_split
# импортируем функцию roc_auc_score()
from sklearn.metrics import roc_auc_score
# создаем обучающий и тестовый массивы признаков и меток
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, 
                                                    stratify=y, random_state=42)
# создаем экземпляр класса StratifiedKFold, 5-блочную
# перекрестную проверку со случайным перемешиванием данных
strat = StratifiedKFold(n_splits=5, shufﬂe=True, random_state=42)
best_score = 0
for max_depth in [4, 6, 8, 10]:
    for max_features in [3, 6, 9, 12]:
        # для каждой из 16 комбинаций значений гиперпараметров создаем экземпляр
        # класса DecisionTreeClassifier - модель дерева CART
        tree = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features,  
                                      random_state=42)
        # для каждой из 16 комбинаций значений гиперпараметров будем строить 5 моделей дерева 
        # CART (5 раз на 4 обучающих блоках перекрестной проверки будем строить модель,
        # на 1 контрольном блоке - проверять, вычисляя AUC)
        scores = cross_val_score(tree, X_train, y_train, scoring='roc_auc', cv=strat)
        # для каждой из 16 комбинаций значений гиперпараметров вычисляем значение AUC, 
        # усредненное по 5 контрольным блокам перекрестной проверки 
        auc_score = np.mean(scores)
        # если получаем максимальное усредненное значение AUC, 
        # сохраняем его и комбинацию значений гиперпараметров
        if auc_score > best_score:
            best_score = auc_score
            best_parameters = {'max_features': max_features, 'max_depth': max_depth}
# строим модель с комбинацией значений гиперпараметров, давшей максимальное 
# усредненное значение AUC, на обучающей выборке
tree_best = DecisionTreeClassifier(**best_parameters, random_state=42)
tree_best.fit(X_train, y_train)
# проверяем качество модели с комбинацией значений гиперпараметров, 
# давшей максимальное усредненное значение AUC, на тестовой выборке
test_score = roc_auc_score(y_test, tree_best.predict_proba(X_test)[:, 1])
print('Наилучшие значения гиперпараметров: ', best_parameters)
print('Лучшее усредненное значение AUC cv: {:.2f}'.format(best_score))
print('AUC модели с наилучшими значениями гиперпараметров на тестовой выборке: {:.2f}'.format(test_score))

Наилучшие значения гиперпараметров:  {'max_features': 9, 'max_depth': 4}
Лучшее усредненное значение AUC cv: 0.79
AUC модели с наилучшими значениями гиперпараметров на тестовой выборке: 0.78


## Комбинированная проверка с помощью класса GridSearchCV

In [366]:
# создаем экземпляр класса DecisionTreeClassifier
tree_grid = DecisionTreeClassifier(random_state=42)
# импортируем класс GridSearchCV
from sklearn.model_selection import GridSearchCV
# задаем сетку параметров, будем перебирать разные значения штрафа
param_grid = {'max_depth': [4, 6, 8, 10], 
              'max_features': [3, 6, 9, 12]}
# создаем экземпляр класса GridSearchCV
grid_search = GridSearchCV(tree_grid, 
                           param_grid, 
                           scoring='roc_auc', 
                           return_train_score=True,
                           n_jobs=-1, cv=strat)
# запускаем решетчатый поиск
grid_search.fit(X_train, y_train)
# проверяем качество модели с комбинацией значений гиперпараметров, 
# давшей максимальное усредненное значение AUC, на тестовой выборке
test_score = roc_auc_score(y_test, grid_search.predict_proba(X_test)[:, 1])
# смотрим результаты решетчатого поиска
print('Наилучшие значения гиперпараметров: {}'.format(grid_search.best_params_))
print('Лучшее усредненное значение AUC cv: {:.2f}'.format(grid_search.best_score_))
print('AUC модели с наилучшими значениями гиперпараметров на тестовой выборке: {:.2f}'.format(test_score))

Наилучшие значения гиперпараметров: {'max_depth': 4, 'max_features': 9}
Лучшее усредненное значение AUC cv: 0.79
AUC модели с наилучшими значениями гиперпараметров на тестовой выборке: 0.78


## Разбиение на обучающую и тестовую выборки, учитывающее временную структуру данных

In [428]:
# записываем CSV-файл в объект DataFrame
timeseries_data = pd.read_csv('Data/Flats.csv', 
                              encoding='UTF-8', 
                              sep=';', 
                              decimal=',')

In [429]:
# выводим первые 5 наблюдений датафрейма
timeseries_data.head()

,Rooms_Number,Street,House_Number,Metro_m,Stor,Storeys,Wall,Space_Total,Value_abs,Date_Create
0,1,Оловозаводская,12/1,6340,1,6,Кирпич,32.5,1450000,22.08.2016
1,1,Оловозаводская,12/1,6340,2,6,Кирпич,33.8,1650000,30.01.2017
2,1,Оловозаводская,12/1,6340,6,6,Кирпич,37.2,2250000,28.10.2014
3,1,Оловозаводская,12/1,6340,6,6,Кирпич,36.4,1960000,19.08.2014
4,1,Оловозаводская,12/1,6340,2,6,Кирпич,35.3,1950000,28.02.2014


In [430]:
# преобразовываем в формат даты
timeseries_data['Date_Create'] = pd.to_datetime(timeseries_data['Date_Create'], 
                                                format='%d.%m.%Y')

In [431]:
# сортируем данные по дате сделки
# (от самой ранней к самой поздней)
timeseries_data = timeseries_data.sort_values(by='Date_Create', 
                                              ascending=1)
timeseries_data

,Rooms_Number,Street,House_Number,Metro_m,Stor,Storeys,Wall,Space_Total,Value_abs,Date_Create
11,2,Сухарная,76/3,3730,7,10,Кирпич,57.20,3300000,2005-07-06
5,1,Оловозаводская,12/1,6340,2,6,Кирпич,34.00,1700000,2011-11-23
12,3,Сухарная,76/3,3730,5,15,Кирпич,76.30,4950000,2013-12-06
16,3,Народная,16/1,3210,1,5,Панель,58.00,2900000,2014-01-30
4,1,Оловозаводская,12/1,6340,2,6,Кирпич,35.30,1950000,2014-02-28
9,2,Сухарная,76/3,3730,14,15,Кирпич,57.10,3850000,2014-04-17
3,1,Оловозаводская,12/1,6340,6,6,Кирпич,36.40,1960000,2014-08-19
2,1,Оловозаводская,12/1,6340,6,6,Кирпич,37.20,2250000,2014-10-28
18,2,Чкалова,70/1,4390,4,9,Кирпич,47.00,3100000,2014-12-01
10,3,Сухарная,76/3,3730,13,15,Кирпич,74.20,5450000,2015-02-11


In [432]:
# разбиваем данные на обучающую и тестовую выборки
timeseries_train = timeseries_data.iloc[0:14]
timeseries_test = timeseries_data.iloc[14:20]

In [433]:
# смотрим обучающую выборку
timeseries_train

,Rooms_Number,Street,House_Number,Metro_m,Stor,Storeys,Wall,Space_Total,Value_abs,Date_Create
11,2,Сухарная,76/3,3730,7,10,Кирпич,57.2,3300000,2005-07-06
5,1,Оловозаводская,12/1,6340,2,6,Кирпич,34.0,1700000,2011-11-23
12,3,Сухарная,76/3,3730,5,15,Кирпич,76.3,4950000,2013-12-06
16,3,Народная,16/1,3210,1,5,Панель,58.0,2900000,2014-01-30
4,1,Оловозаводская,12/1,6340,2,6,Кирпич,35.3,1950000,2014-02-28
9,2,Сухарная,76/3,3730,14,15,Кирпич,57.1,3850000,2014-04-17
3,1,Оловозаводская,12/1,6340,6,6,Кирпич,36.4,1960000,2014-08-19
2,1,Оловозаводская,12/1,6340,6,6,Кирпич,37.2,2250000,2014-10-28
18,2,Чкалова,70/1,4390,4,9,Кирпич,47.0,3100000,2014-12-01
10,3,Сухарная,76/3,3730,13,15,Кирпич,74.2,5450000,2015-02-11


In [434]:
# смотрим тестовую выборку
timeseries_test

,Rooms_Number,Street,House_Number,Metro_m,Stor,Storeys,Wall,Space_Total,Value_abs,Date_Create
15,2,Народная,16/1,3210,1,5,Панель,44.40,1850000,2017-01-30
1,1,Оловозаводская,12/1,6340,2,6,Кирпич,33.80,1650000,2017-01-30
7,1,Шатурская,4,19820,17,17,Кирпич,35.27,2330000,2017-02-15
14,1,Сухарная,76/3,3730,13,15,Кирпич,37.10,2300000,2017-12-06
17,2,Чкалова,70/1,4390,1,9,Кирпич,52.00,2400000,2018-02-21
8,1,Планировочная,37,0,5,10,NaN,33.00,1900000,2018-08-30


In [435]:
# а еще можно так
# train_size = int(len(timeseries_data) * 0.7)
# train, test = timeseries_data[0:train_size], timeseries_data[train_size:len(timeseries_data)]

In [436]:
# записываем CSV-файл в объект DataFrame
mfo_data = pd.read_csv('Data/MFOCredit.csv', sep=';')
# выводим первые 5 наблюдений
mfo_data.head()

,date_start,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,03-Jan-2013,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
1,03-Jan-2013,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет
2,03-Jan-2013,Мужской,25,Пропуск поля,Собственное,Холост,Да,Официальное,Кемеровская область север,Пропуск поля,Нет,Нет
3,03-Jan-2013,Женский,47,Пропуск поля,Собственное,Гражданский брак/женат/замужем,Да,Официальное,Кемеровская область север,Нет,Нет,Нет
4,03-Jan-2013,Мужской,22,Нет,Арендуемое,Гражданский брак/женат/замужем,Нет,Официальное,Кемеровская область север,Да,Да,Нет


In [437]:
# преобразовываем в формат даты
mfo_data['date_start'] = pd.to_datetime(mfo_data['date_start'])
# выводим первые 5 наблюдений
mfo_data.head()

,date_start,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,2013-01-03,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
1,2013-01-03,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет
2,2013-01-03,Мужской,25,Пропуск поля,Собственное,Холост,Да,Официальное,Кемеровская область север,Пропуск поля,Нет,Нет
3,2013-01-03,Женский,47,Пропуск поля,Собственное,Гражданский брак/женат/замужем,Да,Официальное,Кемеровская область север,Нет,Нет,Нет
4,2013-01-03,Мужской,22,Нет,Арендуемое,Гражданский брак/женат/замужем,Нет,Официальное,Кемеровская область север,Да,Да,Нет


In [438]:
# сортируем данные по дате займа
# (от самой ранней к самой поздней)
mfo_data = mfo_data.sort_values(by='date_start', 
                                ascending=1)

In [439]:
# удаляем на месте date_start
mfo_data.drop('date_start', inplace=True, axis=1)

In [440]:
# выполняем разбиение на обучающую и тестовую выборки,
# учитывающее временную структуру
train_size = int(len(mfo_data) * 0.7)
train, test = mfo_data[0:train_size], mfo_data[train_size:len(mfo_data)]

In [441]:
# создаем обучающий массив меток и массив признаков
y_mfo_train = train.pop('delinq60plus')
X_mfo_train = pd.get_dummies(train)

In [442]:
# создаем тестовый массив меток и массив признаков
y_mfo_test = test.pop('delinq60plus')
X_mfo_test = pd.get_dummies(test)

In [443]:
# импортируем класс GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
# создаем экземляр класса GradientBoostingClassifier
boost = GradientBoostingClassifier(random_state=152)
# обучаем модель
boost.fit(X_mfo_train, y_mfo_train)
# оцениваем качество модели на тестовых данных
print('Правильность на тестовой выборке: {:.3f}'.format(
    boost.score(X_mfo_test, y_mfo_test)))

Правильность на тестовой выборке: 0.507


In [444]:
# выполняем случайное разбиение на обучающую и тестовую выборки
X_mfo_tr, X_mfo_tst, y_mfo_tr, y_mfo_tst = train_test_split(mfo_data.drop('delinq60plus', axis=1), 
                                                            mfo_data['delinq60plus'], 
                                                            test_size=0.3, 
                                                            random_state=42)
X_mfo_tr = pd.get_dummies(X_mfo_tr)
X_mfo_tst = pd.get_dummies(X_mfo_tst)

In [445]:
# обучаем модель
boost.fit(X_mfo_tr, y_mfo_tr)
# оцениваем качество модели на тестовых данных
print('Правильность на тестовой выборке: {:.3f}'.format(
    boost.score(X_mfo_tst, y_mfo_tst)))

Правильность на тестовой выборке: 0.665


## Перекрестная проверка, учитывающая временную структуру данных

In [446]:
# импортируем класс TimeSeriesSplit
from sklearn.model_selection import TimeSeriesSplit
# создаем экземпляр класса TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)
# создаем массив меток и массив признаков
y_timeseries = timeseries_data.pop('Value_abs').values
X_timeseries = timeseries_data.values

In [447]:
# взглянем на индексы наблюдений, попавших в обучающий
# и тестовый наборы, по каждой из 3 итераций
for train_index, test_index in tscv.split(X_timeseries):
    print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = X_timeseries[train_index], X_timeseries[test_index]
    y_train, y_test = y_timeseries[train_index], X_timeseries[test_index]
    print('Общее кол-во наблюдений: %d' % (len(X_train) + len(X_test)))
    print('Обучающий набор: %d' % (len(X_train)))
    print('Тестовый набор: %d' % (len(X_test)))
    print('')

TRAIN: [0 1 2 3 4] TEST: [5 6 7 8 9]
Общее кол-во наблюдений: 10
Обучающий набор: 5
Тестовый набор: 5

TRAIN: [0 1 2 3 4 5 6 7 8 9] TEST: [10 11 12 13 14]
Общее кол-во наблюдений: 15
Обучающий набор: 10
Тестовый набор: 5

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14] TEST: [15 16 17 18 19]
Общее кол-во наблюдений: 20
Обучающий набор: 15
Тестовый набор: 5



In [448]:
# фиксируем размер обучающего набора
tscv2 = TimeSeriesSplit(n_splits=3, max_train_size=5)

In [449]:
# взглянем на индексы наблюдений, попавших в обучающий
# и тестовый наборы, по каждой из 3 итераций
for train_index, test_index in tscv2.split(X_timeseries):
    print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = X_timeseries[train_index], X_timeseries[test_index]
    y_train, y_test = y_timeseries[train_index], y_timeseries[test_index]
    print('Общее кол-во наблюдений: %d' % (len(X_train) + len(X_test)))
    print('Обучающий набор: %d' % (len(X_train)))
    print('Тестовый набор: %d' % (len(X_test)))
    print('')

TRAIN: [0 1 2 3 4] TEST: [5 6 7 8 9]
Общее кол-во наблюдений: 10
Обучающий набор: 5
Тестовый набор: 5

TRAIN: [5 6 7 8 9] TEST: [10 11 12 13 14]
Общее кол-во наблюдений: 10
Обучающий набор: 5
Тестовый набор: 5

TRAIN: [10 11 12 13 14] TEST: [15 16 17 18 19]
Общее кол-во наблюдений: 10
Обучающий набор: 5
Тестовый набор: 5



In [450]:
# создаем массив меток и массив признаков
y_mfo = mfo_data.pop('delinq60plus').values
X_mfo = pd.get_dummies(mfo_data).values

In [472]:
# создаем экземпляр класса TimeSeriesSplit
time_cv = TimeSeriesSplit(n_splits=3)

In [474]:
# взглянем на индексы наблюдений, попавших в обучающий
# и тестовый наборы, по каждой из 3 итераций, а также
# посмотрим правильность на разных временных срезах
for train_index, test_index in time_cv.split(X_mfo):
    print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = X_mfo[train_index], X_mfo[test_index]
    y_train, y_test = y_mfo[train_index], y_mfo[test_index]
    print('Общее кол-во наблюдений: %d' % (len(X_train) + len(X_test)))
    print('Обучающий набор: %d' % (len(X_train)))
    print('Тестовый набор: %d' % (len(X_test)))
    boost.fit(X_train, y_train)
    print('Правильность на обучающем наборе: {:.3f}'.format(
        boost.score(X_train, y_train)))
    print('Правильность на тестовом наборе: {:.3f}'.format(
        boost.score(X_test, y_test)))
    print('')

TRAIN: [   0    1    2 ... 8800 8801 8802] TEST: [ 8803  8804  8805 ... 17603 17604 17605]
Общее кол-во наблюдений: 17606
Обучающий набор: 8803
Тестовый набор: 8803
Правильность на обучающем наборе: 0.709
Правильность на тестовом наборе: 0.615

TRAIN: [    0     1     2 ... 17603 17604 17605] TEST: [17606 17607 17608 ... 26406 26407 26408]
Общее кол-во наблюдений: 26409
Обучающий набор: 17606
Тестовый набор: 8803
Правильность на обучающем наборе: 0.668
Правильность на тестовом наборе: 0.515

TRAIN: [    0     1     2 ... 26406 26407 26408] TEST: [26409 26410 26411 ... 35209 35210 35211]
Общее кол-во наблюдений: 35212
Обучающий набор: 26409
Тестовый набор: 8803
Правильность на обучающем наборе: 0.640
Правильность на тестовом наборе: 0.571



## Бутстреп

In [138]:
# импортируем необходимые функции
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
# конкатенируем массив признаков и массив меток
data = pd.concat([X, y], axis=1)
# преобразовываем датафрейм в массив NumPy
values = data.values
# задаем количество итераций
n_iterations = 100
n_size = int(len(data) * 0.50)
# создаем список, в котором будем 
# хранить значения правильности
acc_stats = list()
# создаем список, в котором будем 
# хранить значения AUC
auc_stats = list()
# запускаем бутстреп
for i in range(n_iterations):
    # подготавливаем обучающий и тестовый наборы
    train = resample(values, n_samples=n_size)
    test = np.array([x for x in values if x.tolist() not in train.tolist()])
    # обучаем модель дерева
    model = DecisionTreeClassifier(random_state=152)
    model.fit(train[:,:-1], train[:,-1])
    # оцениваем правильность модели
    predictions = model.predict(test[:,:-1])
    acc_score = accuracy_score(test[:,-1], predictions)
    acc_stats.append(acc_score)
    # оцениваем AUC модели
    probabilities = model.predict_proba(test[:,:-1])[:, 1]
    auc_score = roc_auc_score(test[:,-1], probabilities)
    auc_stats.append(auc_score)
# печатаем среднее значение правильности
print("Среднее значение правильности: {:.2f}".format(np.mean(acc_stats)))
# печатаем среднее значение AUC
print("Среднее значение AUC: {:.2f}".format(np.mean(auc_stats)))

Среднее значение правильности: 0.67
Среднее значение AUC: 0.64


## Доверительный интервал

### Асимптотический метод

#### Учимся вычислять асимптотический 90%-ный доверительный интервал среднего

In [14]:
# импортируем модуль random
import random
# задаем стартовое значение
# генератора случайных чисел
random.seed(444)
# генеририруем 50 случайных чисел
# в диапазоне от 45 до 100
income = np.random.uniform(45, 100, size=(50,))

In [15]:
# вычисляем среднее значение дохода
mean_income = np.mean(income)
mean_income

73.7432251233859

In [16]:
# записываем информацию о размере выборки
N = 50

In [17]:
# вычисляем предел погрешности 
err = 1.645 * (np.std(income) / np.sqrt(N))

In [18]:
# вычисляем нижнюю границу 90%-ного
# доверительного интервала 
mean_income - err

70.0359399671963

In [19]:
# вычисляем верхнюю границу 90%-ного
# доверительного интервала 
mean_income + err

77.4505102795755

### Бутстреп-метод

#### Учимся вычислять бутстрепированный 95%-ный доверительный интервал среднего

In [20]:
# импортируем модуль stats
from scipy import stats

# пишем функцию, извлекающую бутстреп-выборки
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

# пишем функцию, вычисляющую нижнюю и верхнюю границы доверительного 
# интервала для полученного распределения оценок
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

# пишем итоговую функцию, в которой объединяем первые две
def bootstrap_conf_int(data, stat_func, alpha=0.05, n_samples=1000):
    ''' 
    a = np.random.normal(size=1000)
    conf_int(a, np.median)
    '''
    scores = [stat_func(sample) for sample in get_bootstrap_samples(data, n_samples)]
    return stat_intervals(scores, alpha)

In [21]:
# вычисляем бутстрепированный 95%-ный 
# доверительный интервал среднего
bootstrap_conf_int(income, np.mean, alpha=0.05, n_samples=1000)

array([69.2450439 , 77.96877399])

### Доверительный интервал метрики качества (на примере AUC)

#### Учимся вычислять бутстрепированный 95%-ный доверительный интервал AUC и правильности

In [22]:
# вычисляем бутстрепированные доверительные интервалы
# правильности и AUC
alpha = 0.95
# задаем процентиль 2,5
p = ((1.0 - alpha) / 2.0) * 100
# вычисляем нижнюю границу доверительного интервала правильности
lower_acc = max(0.0, np.percentile(acc_stats, p))
# вычисляем нижнюю границу доверительного интервала AUC
lower_auc = max(0.0, np.percentile(auc_stats, p))
# задаем процентиль 97,5
p = (alpha + ((1.0 - alpha) / 2.0)) * 100
# вычисляем верхнюю границу доверительного интервала правильности
upper_acc = min(1.0, np.percentile(acc_stats, p))
# вычисляем верхнюю границу доверительного интервала AUC
upper_auc = min(1.0, np.percentile(auc_stats, p))
# печатаем доверительный интервал правильности
print("%.1f доверительный интервал правильности %.1f%% и %.1f%%" % (
    alpha * 100, lower_acc * 100, upper_acc * 100))
# печатаем доверительный интервал AUC
print("%.1f доверительный интервал AUC %.1f%% и %.1f%%" % (
    alpha * 100, lower_auc * 100, upper_auc * 100))

95.0 доверительный интервал правильности 63.8% и 70.0%
95.0 доверительный интервал AUC 61.2% и 67.7%
